In [ ]:
torch.cuda.empty_cache()

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# Set device
device = torch.device("cuda")

# Define data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load the dataset
train_dataset = datasets.ImageFolder('/home/bvrit/datasets/datasign2Split/train', transform=transform)
val_dataset = datasets.ImageFolder('/home/bvrit/datasets/datasign2Split/validate', transform=transform)
test_dataset = datasets.ImageFolder('/home/bvrit/datasets/datasign2Split/test', transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

# Define the ResNet-18 model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,4)  # Assuming 3 labels/classes: A, B, Y

# Move the model to the device
model = model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Print training and validation statistics
    epoch_loss = running_loss / len(train_loader)
    epoch_val_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Loss: {epoch_loss:.4f}, "
          f"Val Loss: {epoch_val_loss:.4f}, "
          f"Accuracy: {accuracy:.2f}%")
    torch.cuda.empty_cache()

# Testing loop
model.eval()
test_loss = 0.0
correct = 0
total = 0
predictions = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())  # Collect predictions

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
accuracy = 100 * correct / total

# Map predicted indices to class labels
label_map = {0: '0', 1: '100', 2: '108',3:'112'}
predicted_labels = [label_map[prediction] for prediction in predictions]

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
print("Predicted Labels:", predicted_labels)

# Save the trained model
torch.save(model.state_dict(), '2resnet_model.pth')


/home/bvrit/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bvrit/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/15], Loss: 0.0522, Val Loss: 0.2193, Accuracy: 92.61%
Epoch [2/15], Loss: 0.0152, Val Loss: 0.0122, Accuracy: 99.67%
Epoch [3/15], Loss: 0.0083, Val Loss: 0.0042, Accuracy: 99.94%
Epoch [4/15], Loss: 0.0024, Val Loss: 0.0004, Accuracy: 100.00%
Epoch [5/15], Loss: 0.0009, Val Loss: 0.0004, Accuracy: 100.00%
Epoch [6/15], Loss: 0.0257, Val Loss: 0.0005, Accuracy: 100.00%
Epoch [7/15], Loss: 0.0107, Val Loss: 0.0030, Accuracy: 99.89%
Epoch [8/15], Loss: 0.0016, Val Loss: 0.0001, Accuracy: 100.00%
Epoch [9/15], Loss: 0.0004, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [10/15], Loss: 0.0001, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [11/15], Loss: 0.0001, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [12/15], Loss: 0.0000, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [13/15], Loss: 0.0000, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [14/15], Loss: 0.0000, Val Loss: 0.0000, Accuracy: 100.00%
Epoch [15/15], Loss: 0.0000, Val Loss: 0.0000, Accuracy: 100.00%
Test Loss: 0.0000, Test Accuracy: 100.